## Exercise 3

We'll first just copy relevant code from the last two exercises to start where we left off.

In [1]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

# Relevant code from exercise 1.1
product_data = pd.read_csv('https://github.com/Mixtape-Sessions/Demand-Estimation/raw/main/Exercises/Data/products.csv')

# Relevant code from exercise 1.2
product_data['market_size'] = product_data['city_population'] * 90
product_data['market_share'] = product_data['servings_sold'] / product_data['market_size']

# Relevant code from exercise 1.4
product_data = product_data.rename(columns={
    'market': 'market_ids',
    'product': 'product_ids',
    'market_share': 'shares',
    'price_per_serving': 'prices',
})

# Relevant code from exercise 1.6
first_stage = smf.ols('prices ~ 0 + price_instrument + C(market_ids) + C(product_ids)', product_data)
first_stage_results = first_stage.fit(cov_type='HC0')
product_data = product_data.rename(columns={'price_instrument': 'demand_instruments0'})
iv_problem = pyblp.Problem(pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), product_data)
iv_results = iv_problem.solve(method='1s')

# Relevant code from exercise 1.7
counterfactual_market = 'C01Q2'
counterfactual_data = product_data.loc[product_data['market_ids'] == counterfactual_market, ['product_ids', 'mushy', 'prices', 'shares']]
counterfactual_data['new_prices'] = counterfactual_data['prices']
counterfactual_data.loc[counterfactual_data['product_ids'] == 'F1B04', 'new_prices'] /= 2
counterfactual_data['new_shares'] = iv_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['iv_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']

# Relevant code from exercise 2.1
demographic_data = pd.read_csv('https://github.com/Mixtape-Sessions/Demand-Estimation/raw/main/Exercises/Data/demographics.csv')
demographic_data = demographic_data.rename(columns={'market': 'market_ids'})
demographic_data['log_income'] = np.log(demographic_data['quarterly_income'])
demographic_variation = demographic_data.groupby('market_ids', as_index=False).agg(**{
    'log_income_mean': ('log_income', 'mean'),
    'log_income_std': ('log_income', 'std'),
})

# Relevant code from exercise 2.2
agent_data = demographic_data[['market_ids', 'log_income']].groupby('market_ids', as_index=False).sample(n=1000, replace=True, random_state=0)
agent_data[['nodes0', 'nodes1', 'nodes2']] = np.random.default_rng(seed=0).normal(size=(len(agent_data), 3))
agent_data['weights'] = 1 / agent_data.groupby('market_ids').transform('size')
product_data = product_data.merge(demographic_variation[['market_ids', 'log_income_mean']], on='market_ids')
product_data['demand_instruments1'] = product_data['log_income_mean'] * product_data['mushy']
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('0 + mushy'))
agent_formulation = pyblp.Formulation('0 + log_income')
mushy_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
optimization = pyblp.Optimization('trust-constr', {'gtol': 1e-8, 'xtol': 1e-8})
mushy_results = mushy_problem.solve(sigma=0, pi=1, method='1s', optimization=optimization)

# Relevant code from exercise 2.4
counterfactual_data['new_shares'] = mushy_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['mushy_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']

# Relevant code from exercise 2.5
product_data['predicted_prices'] = first_stage_results.fittedvalues
product_data['demand_instruments2'] = product_data['log_income_mean'] * product_data['predicted_prices']
compute_differentiation = lambda x: np.sum((x.values[:, None] - x.values[None, :])**2, axis=1)
product_data['demand_instruments3'] = product_data.groupby('market_ids')['predicted_prices'].transform(compute_differentiation)
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('0 + mushy + prices'))
agent_formulation = pyblp.Formulation('0 + log_income')
rc_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
rc_results = rc_problem.solve(
    sigma=[
        [0, 0],
        [0, 1],
    ], 
    pi=[
        [0.2],
        [1],
    ], 
    method='1s', 
    optimization=optimization,
)

# Relevant code from exercise 2.6
counterfactual_data['new_shares'] = rc_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['rc_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']

### 1. Use the income statistic to match a parameter on log income

First, we'll add a constant to our `X2` formulation.

In [2]:
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('1 + mushy + prices'))
agent_formulation = pyblp.Formulation('0 + log_income')
micro_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
micro_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     3     1    4     2  

Formulations:
       Column Indices:             0         1      2   
-----------------------------  ----------  -----  ------
 X1: Linear Characteristics      prices                 
X2: Nonlinear Characteristics      1       mushy  prices
       d: Demographics         log_income               

Next we'll define our micro dataset.

In [3]:
survey_markets = ['C01Q1', 'C01Q2']
compute_income_weights = lambda t, p, a: np.einsum('i,j', np.ones(a.size), np.ones(p.size))
income_dataset = pyblp.MicroDataset("Income Survey", 100, compute_income_weights, market_ids=survey_markets)
income_dataset

Income Survey: 100 Observations in 2 Markets

On it, we'll define our micro part.

In [4]:
compute_income_values = lambda t, p, a: np.einsum('i,j', a.demographics[:, 0], np.ones(p.size))
income_part = pyblp.MicroPart("E[log_income_i | j > 0]", income_dataset, compute_income_values)
income_part

E[log_income_i | j > 0] on Income Survey: 100 Observations in 2 Markets

Using this, we can define our micro moment.

In [5]:
income_moment = pyblp.MicroMoment("E[log_income_i | j > 0]", 7.9, income_part)
income_moment

E[log_income_i | j > 0]: +7.90E+00 (E[log_income_i | j > 0] on Income Survey: 100 Observations in 2 Markets)

We can use this in estimation to pin down our new parameter on income alone.

In [6]:
pyblp.options.verbose = True
micro_results = micro_problem.solve(
    sigma=[
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 6],
    ], 
    pi=[
        [1],
        [0.1],
        [-6],
    ], 
    method='1s', 
    optimization=optimization, 
    micro_moments=[income_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed           Moment                    Part               Dataset     Observations  Markets
---------  -----------------------  -----------------------  -------------  ------------  -------
+7.90E+00  E[log_income_i | j > 0]  E[log_income_i | j > 0]  Income Survey      100          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +0.00E+00                        |    1     +1.00E+00 
mushy   +0.00E+00  +0.00E+00             |  mushy   +1.00E-01 
prices  +0.00E+00  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +0.00E+00                        |    1       -INF    
mushy   +0.00E+00  +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.0

The new parameter estimate is not significantly different from zero, suggesting that our original assumption that it was zero was not too bad. This is not at all guaranteed, we may have just been lucky! (Or this imagined statistic may have been chosen by the instructor to make this happen.)

### 2. Use the diversion statistics to estimate unobserved preference heterogeneity for a constant and mushy

Let's first define our new micro dataset.

In [7]:
compute_diversion_weights = lambda t, p, a: np.einsum('i,j,k', np.ones(a.size), np.ones(p.size), np.ones(1 + p.size))
diversion_dataset = pyblp.MicroDataset("Diversion Survey", 200, compute_diversion_weights, market_ids=survey_markets)
diversion_dataset

Diversion Survey: 200 Observations in 2 Markets

Now let's define our first moment for matching outside diversion.

In [8]:
compute_outside_values = lambda t, p, a: np.einsum('i,j,k', np.ones(a.size), np.ones(p.size), np.r_[1, np.zeros(p.size)])
outside_part = pyblp.MicroPart("P(k = 0 | j > 0)", diversion_dataset, compute_outside_values)
outside_moment = pyblp.MicroMoment("P(k = 0 | j > 0)", 0.28, outside_part)
outside_moment

P(k = 0 | j > 0): +2.80E-01 (P(k = 0 | j > 0) on Diversion Survey: 200 Observations in 2 Markets)

Let's also define our second moment for matching mushy diversion.

In [9]:
compute_mushy_values = lambda t, p, a: np.einsum('i,j,k', np.ones(a.size), p.X2[:, 1], np.r_[0, p.X2[:, 1]])
mushy_part = pyblp.MicroPart("P(mushy_j and mushy_k | j > 0)", diversion_dataset, compute_mushy_values)
mushy_moment = pyblp.MicroMoment("P(mushy_j and mushy_k | j > 0)", 0.31, mushy_part)
mushy_moment

P(mushy_j and mushy_k | j > 0): +3.10E-01 (P(mushy_j and mushy_k | j > 0) on Diversion Survey: 200 Observations in 2 Markets)

Then we can re-optimize with our new micro moments, choosing some initial values for our new parameters.

In [10]:
pyblp.options.verbose = True
micro_results = micro_problem.solve(
    sigma=[
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 6],
    ], 
    pi=[
        [-0.3],
        [0.1],
        [-6],
    ], 
    method='1s', 
    optimization=optimization, 
    micro_moments=[income_moment, outside_moment, mushy_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed               Moment                           Part                   Dataset       Observations  Markets
---------  ------------------------------  ------------------------------  ----------------  ------------  -------
+7.90E+00     E[log_income_i | j > 0]         E[log_income_i | j > 0]       Income Survey        100          2   
+2.80E-01         P(k = 0 | j > 0)                P(k = 0 | j > 0)         Diversion Survey      200          2   
+3.10E-01  P(mushy_j and mushy_k | j > 0)  P(mushy_j and mushy_k | j > 0)  Diversion Survey      200          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +1.00E+00                        |    1     -3.00E-01 
mushy   +0.00E+00  +1.00E+00             |  mushy   +1.00E-01 
prices  +0.00E+00  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bo

All the standard optimization checks look fine. The new estimates suggest that there is a good amount of unobserved preference heterogeneity for mushy, and some for the constant characteristic (i.e. the outside good).

### 3. Evaluate changes to the price cut counterfactual

Finally, let's re-run the price counterfactual with our more flexible model.

In [11]:
counterfactual_data['new_shares'] = micro_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['micro_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,mushy,prices,shares,new_prices,new_shares,iv_change,mushy_change,rc_change,micro_change
24,F1B04,1,0.078,6.443e-03,0.039,2.667e-02,223.638,223.522,285.128,314.014
25,F1B06,1,0.141,1.413e-01,0.141,1.343e-01,-1.450,-1.478,-1.621,-4.934
26,F1B07,1,0.073,8.789e-02,0.073,8.235e-02,-1.450,-1.478,-1.850,-6.303
27,F1B09,0,0.077,6.621e-03,0.077,6.572e-03,-1.450,-1.438,-1.808,-0.750
28,F1B11,0,0.167,5.427e-02,0.167,5.401e-02,-1.450,-1.438,-1.496,-0.482
29,F1B13,0,0.092,2.198e-02,0.092,2.182e-02,-1.450,-1.438,-1.759,-0.700
30,F1B17,1,0.154,1.055e-02,0.154,1.005e-02,-1.450,-1.478,-1.575,-4.696
31,F1B30,0,0.150,1.310e-03,0.150,1.303e-03,-1.450,-1.438,-1.556,-0.526
32,F1B45,0,0.147,1.052e-02,0.147,1.047e-02,-1.450,-1.438,-1.568,-0.534
33,F2B05,0,0.099,5.907e-02,0.099,5.867e-02,-1.450,-1.438,-1.735,-0.677


Substitution and cannibalization now looks much more reasonable. We see much more substitution within mushy cereals, which makes sense because if the price drops for a mushy cereal, we would expect mainly consumers of similar cereals to substitute to it.

## Supplemental Questions

### 1. See how your market size assumption affects results

First let's estimate the model without the parameter in $\Sigma$ on the constant, effectively assuming zero unobserved preference heterogeneity for the outside option. We'll also drop its micro moment.

In [12]:
pyblp.options.verbose = True
restricted_results = micro_problem.solve(
    sigma=[
        [0, 0, 0],
        [0, 4, 0],
        [0, 0, 6],
    ], 
    pi=[
        [-0.3],
        [0.1],
        [-6],
    ], 
    method='1s', 
    optimization=optimization, 
    micro_moments=[income_moment, mushy_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed               Moment                           Part                   Dataset       Observations  Markets
---------  ------------------------------  ------------------------------  ----------------  ------------  -------
+7.90E+00     E[log_income_i | j > 0]         E[log_income_i | j > 0]       Income Survey        100          2   
+3.10E-01  P(mushy_j and mushy_k | j > 0)  P(mushy_j and mushy_k | j > 0)  Diversion Survey      200          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +0.00E+00                        |    1     -3.00E-01 
mushy   +0.00E+00  +4.00E+00             |  mushy   +1.00E-01 
prices  +0.00E+00  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  --

Now let's initialize a new problem with a market size that is twice as large as before.

In [13]:
alt_product_data = product_data.copy()
alt_product_data['market_size'] *= 2
alt_product_data['shares'] = alt_product_data['servings_sold'] / alt_product_data['market_size']
alt_problem = pyblp.Problem(product_formulations, alt_product_data, agent_formulation, agent_data)

Let's first estimate the same restricted version of this model without unobserved preference heterogeneity for the outside option.

In [14]:
pyblp.options.verbose = True
alt_restricted_results = alt_problem.solve(
    sigma=[
        [0, 0, 0],
        [0, 4, 0],
        [0, 0, 6],
    ], 
    pi=[
        [-0.3],
        [0.1],
        [-6],
    ], 
    method='1s', 
    optimization=optimization, 
    micro_moments=[income_moment, mushy_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed               Moment                           Part                   Dataset       Observations  Markets
---------  ------------------------------  ------------------------------  ----------------  ------------  -------
+7.90E+00     E[log_income_i | j > 0]         E[log_income_i | j > 0]       Income Survey        100          2   
+3.10E-01  P(mushy_j and mushy_k | j > 0)  P(mushy_j and mushy_k | j > 0)  Diversion Survey      200          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +0.00E+00                        |    1     -3.00E-01 
mushy   +0.00E+00  +4.00E+00             |  mushy   +1.00E-01 
prices  +0.00E+00  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  --

Now we'll add back unobserved preference heterogeneity for the outside option and the outside diversion ratio we use to estimate it.

In [15]:
pyblp.options.verbose = True
alt_unrestricted_results = alt_problem.solve(
    sigma=[
        [2, 0, 0],
        [0, 4, 0],
        [0, 0, 6],
    ], 
    pi=[
        [-0.3],
        [0.1],
        [-6],
    ], 
    method='1s', 
    optimization=optimization, 
    micro_moments=[income_moment, outside_moment, mushy_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed               Moment                           Part                   Dataset       Observations  Markets
---------  ------------------------------  ------------------------------  ----------------  ------------  -------
+7.90E+00     E[log_income_i | j > 0]         E[log_income_i | j > 0]       Income Survey        100          2   
+2.80E-01         P(k = 0 | j > 0)                P(k = 0 | j > 0)         Diversion Survey      200          2   
+3.10E-01  P(mushy_j and mushy_k | j > 0)  P(mushy_j and mushy_k | j > 0)  Diversion Survey      200          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +2.00E+00                        |    1     -3.00E-01 
mushy   +0.00E+00  +4.00E+00             |  mushy   +1.00E-01 
prices  +0.00E+00  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bo

Let's run a counterfactual in which we double the prices of all inside goods for all three sets of results.

In [16]:
alt_counterfactual_data = counterfactual_data.copy()
alt_counterfactual_data['alt_shares'] = alt_product_data.loc[alt_product_data['market_ids'] == counterfactual_market, 'shares']
alt_counterfactual_data['restricted_shares'] = restricted_results.compute_shares(market_id=counterfactual_market, prices=alt_counterfactual_data['new_prices'])
alt_counterfactual_data['alt_restricted_shares'] = alt_restricted_results.compute_shares(market_id=counterfactual_market, prices=alt_counterfactual_data['new_prices'])
alt_counterfactual_data['alt_unrestricted_shares'] = alt_unrestricted_results.compute_shares(market_id=counterfactual_market, prices=alt_counterfactual_data['new_prices'])
alt_counterfactual_data = pd.concat([alt_counterfactual_data, pd.DataFrame([{
    'shares': 1 - alt_counterfactual_data['shares'].sum(),
    'alt_shares': 1 - alt_counterfactual_data['alt_shares'].sum(),
    'restricted_shares': 1 - alt_counterfactual_data['restricted_shares'].sum(),
    'alt_restricted_shares': 1 - alt_counterfactual_data['alt_restricted_shares'].sum(),
    'alt_unrestricted_shares': 1 - alt_counterfactual_data['alt_unrestricted_shares'].sum(),
}])])
alt_counterfactual_data['restricted_change'] = 100 * (alt_counterfactual_data['restricted_shares'] - alt_counterfactual_data['shares']) / alt_counterfactual_data['shares']
alt_counterfactual_data['alt_restricted_change'] = 100 * (alt_counterfactual_data['alt_restricted_shares'] - alt_counterfactual_data['alt_shares']) / alt_counterfactual_data['alt_shares']
alt_counterfactual_data['alt_unrestricted_change'] = 100 * (alt_counterfactual_data['alt_unrestricted_shares'] - alt_counterfactual_data['alt_shares']) / alt_counterfactual_data['alt_shares']
alt_counterfactual_data[['product_ids', 'mushy', 'prices', 'new_prices', 'restricted_change', 'alt_restricted_change', 'alt_unrestricted_change']]

,product_ids,mushy,prices,new_prices,restricted_change,alt_restricted_change,alt_unrestricted_change
24,F1B04,1.0,0.078,0.039,327.918,329.669,308.688
25,F1B06,1.0,0.141,0.141,-5.145,-5.225,-4.821
26,F1B07,1.0,0.073,0.073,-6.585,-6.212,-5.885
27,F1B09,0.0,0.077,0.077,-0.521,-0.225,-0.638
28,F1B11,0.0,0.167,0.167,-0.336,-0.142,-0.419
29,F1B13,0.0,0.092,0.092,-0.486,-0.209,-0.597
30,F1B17,1.0,0.154,0.154,-4.902,-5.055,-4.625
31,F1B30,0.0,0.150,0.150,-0.365,-0.155,-0.455
32,F1B45,0.0,0.147,0.147,-0.371,-0.158,-0.462
33,F2B05,0.0,0.099,0.099,-0.470,-0.202,-0.578


The last row measures percent changes to the outside share. When we double the market size, we are effectively increasing the quality of the outside option, so we have less substitution away from it when an inside good becomes cheaper. When we match our outside diversion ratio, we get a different outside diversion estimate pinned down by this micro moment. We estimate even less substitution away from the outside option, would could be a combination of at least two things. First, with this extra parameter we estimate slightly less elastic demand, so we get less substitution in general, and second, the "right" market size may have been a bit larger than the one we originally assumed.

### 2. Simulate some micro data and use it to match optimal micro moments

First let's simulate some fake micro data, just to demonstrate how we'd use it if we actually observed a full micro dataset.

In [17]:
micro_data = pd.DataFrame(pyblp.data_to_dict(micro_results.simulate_micro_data(income_dataset, seed=0)))
micro_data

,micro_ids,market_ids,agent_indices,choice_indices
0,0,C01Q1,47,13
1,1,C01Q1,401,4
2,2,C01Q1,152,9
3,3,C01Q1,39,12
4,4,C01Q2,797,1
...,...,...,...,...
95,95,C01Q2,359,4
96,96,C01Q1,124,18
97,97,C01Q2,39,23
98,98,C01Q1,641,12


PyBLP simulates data from the specifified micro dataset with the following columns: `micro_ids` just increment from `0` to one minus the number of configured `observations` in the micro dataset, `market_ids` are the markets of the observations, `agent_indices` are the row indices (starting from `0`) of the drawn agent types in that market's agent data, and `choice_indices` are the row indices (starting from `0`) of the drawn choices in that market's product data.

In practice, because we have unobserved preference heterogeneity, our micro dataset wouldn't actually contain information about the full individual type $i$ represented by `agent_indices`. Instead, we would just observe income. Let's merge in log income and drop the unobserved `agent_indices`.

In [18]:
agent_data['agent_indices'] = agent_data.groupby('market_ids').cumcount()
micro_data = micro_data.merge(agent_data[['market_ids', 'agent_indices', 'log_income']], on=['market_ids', 'agent_indices'])
micro_data = micro_data.drop(columns='agent_indices')
micro_data

,micro_ids,market_ids,choice_indices,log_income
0,0,C01Q1,13,6.423
1,1,C01Q1,4,7.914
2,2,C01Q1,9,6.423
3,3,C01Q1,12,8.587
4,4,C01Q2,1,7.754
...,...,...,...,...
95,95,C01Q2,4,7.517
96,96,C01Q1,18,7.763
97,97,C01Q2,23,8.186
98,98,C01Q1,12,8.396


When computing scores, PyBLP needs to know how to integrate over each observation's unobserved preference heterogeneity. Similar to how in `agent_data` we have nodes and weights for each market, here we can draw similar nodes and weights for each observation.

Now we can compute the score for each observation.

In [21]:
micro_scores = micro_results.compute_micro_scores(income_dataset, micro_data)
len(micro_scores)

KeyError: 'Since integration is None, micro_data must have nodes.'

The elements in this list of scores correspond to the nonlinear parameters.

In [ ]:
micro_results.theta_labels

['1 x 1',
 'mushy x mushy',
 'prices x prices',
 '1 x log_income',
 'mushy x log_income',
 'prices x log_income']

Each element has a score for each micro observation.

In [ ]:
micro_scores[0].shape

(100,)

To form our optimal micro moments, we also need scores for each possible market-individual type-choice combination $(t, i, j)$ covered by the micro dataset. We'll let PyBLP replicate each consumer type by a number of Monte Carlo draws, like we did above. (We also could have used a similar `integration` argument when computing micro data instead of manually drawing unobserved preference heterogeneity.)

In [ ]:
agent_scores = micro_results.compute_agent_scores(income_dataset, integration=pyblp.Integration('monte_carlo', 1_000, {'seed': 0}))
len(agent_scores)

6

The elements in this list correspond to the same parameters above. But this time, each gives a mapping from market IDs $t$ covered by the micro dataset to scores for each consumer type-choice combination $(i, t)$.

In [ ]:
list(agent_scores[0].keys())

['C01Q1', 'C01Q2']

In [ ]:
agent_scores[0]['C01Q1'].shape

(1000, 24)

With our scores in hand, we can form our optimal micro moments. Note that when dynamically defining a `lambda` function like this, we need to add an extra default argument for the dictionary we are using to "fix" it with the function. Otherwise the function will just use the last `agent_scores_m` in the loop. Alternatively, we could use the `def` approach to defining a function, where we wouldn't need to add an extra argument.

In [ ]:
optimal_micro_moments = []
for m, (label_m, micro_scores_m, agent_scores_m) in enumerate(zip(micro_results.theta_labels, micro_scores, agent_scores)):
    optimal_micro_moments.append(pyblp.MicroMoment(
        name=f"Score for {label_m}",
        value=micro_scores_m.mean(),
        parts=pyblp.MicroPart(
            name=f"Score for {label_m}",
            dataset=income_dataset,
            compute_values=lambda t, p, a, v=agent_scores_m: v[t],
        ),
    ))

optimal_micro_moments

[Score for 1 x 1: -9.55E-04 (Score for 1 x 1 on Income Survey: 100 Observations in 2 Markets),
 Score for mushy x mushy: -1.43E-04 (Score for mushy x mushy on Income Survey: 100 Observations in 2 Markets),
 Score for prices x prices: -1.15E-03 (Score for prices x prices on Income Survey: 100 Observations in 2 Markets),
 Score for 1 x log_income: -1.90E-02 (Score for 1 x log_income on Income Survey: 100 Observations in 2 Markets),
 Score for mushy x log_income: -5.53E-03 (Score for mushy x log_income on Income Survey: 100 Observations in 2 Markets),
 Score for prices x log_income: -1.94E-03 (Score for prices x log_income on Income Survey: 100 Observations in 2 Markets)]

One approach would be to replace the single micro moment we originally based on this dataset with these six optimal micro moments. However, this would give an overidentified model, which is a bit harder to work with, and in any case we don't expect the "Income" micro data to credibly identify parameters in $\Sigma$. To keep our estimates comparible with those from before, we'll just replace the old sub-optimal micro moment with its optimal one.

In [ ]:
pyblp.options.verbose = True
optimal_results = micro_problem.solve(
    sigma=[
        [2, 0, 0],
        [0, 4, 0],
        [0, 0, 6],
    ], 
    pi=[
        [-0.3],
        [0.1],
        [-6],
    ], 
    method='1s', 
    optimization=optimization, 
    micro_moments=[optimal_micro_moments[micro_results.theta_labels.index('1 x log_income')], outside_moment, mushy_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed               Moment                           Part                   Dataset       Observations  Markets
---------  ------------------------------  ------------------------------  ----------------  ------------  -------
-1.90E-02     Score for 1 x log_income        Score for 1 x log_income      Income Survey        100          2   
+2.80E-01         P(k = 0 | j > 0)                P(k = 0 | j > 0)         Diversion Survey      200          2   
+3.10E-01  P(mushy_j and mushy_k | j > 0)  P(mushy_j and mushy_k | j > 0)  Diversion Survey      200          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +2.00E+00                        |    1     -3.00E-01 
mushy   +0.00E+00  +4.00E+00             |  mushy   +1.00E-01 
prices  +0.00E+00  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bo

Results are fairly similar. This is expected because we simulated these micro data at our old estimates, so we'd expect to recover new estimates that look like the old ones. The small micro data size likely accounts for many of the differences.

### 3. Use a within-firm diversion ratio to estimate a nesting parameter

Let's re-create the problem after defining nests equal to firm IDs.

In [ ]:
product_data['nesting_ids'] = product_data['product_ids'].str[:2]
rcnl_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
rcnl_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED    H 
---  ----  -----  ----  ----  ---  ----  ----  ---
94   2256  94000   1     3     1    4     2     5 

Formulations:
       Column Indices:             0         1      2   
-----------------------------  ----------  -----  ------
 X1: Linear Characteristics      prices                 
X2: Nonlinear Characteristics      1       mushy  prices
       d: Demographics         log_income               

Note that there are now $H = 5$ nests equal to the number of firms. Let's define a new micro moment on our diversion survey that matches a share measuring within-firm diversion.

In [ ]:
compute_firm_values = lambda t, p, a: np.ones((a.size, p.size, 1 + p.size)) * (p.nesting_ids == np.c_[[''], p.nesting_ids.T])
firm_part = pyblp.MicroPart("P(firm_j = firm_k | j > 0)", diversion_dataset, compute_firm_values)
firm_moment = pyblp.MicroMoment("P(firm_j = firm_k | j > 0)", 0.35, firm_part)
firm_moment

P(firm_j = firm_k | j > 0): +3.50E-01 (P(firm_j = firm_k | j > 0) on Diversion Survey: 200 Observations in 2 Markets)

Let's use this to estimate a nesting parameter.

In [ ]:
pyblp.options.verbose = True
rcnl_results = rcnl_problem.solve(
    sigma=[
        [2, 0, 0],
        [0, 4, 0],
        [0, 0, 6],
    ], 
    pi=[
        [-0.3],
        [0.1],
        [-6],
    ],
    rho=0.1,
    method='1s', 
    optimization=optimization, 
    micro_moments=[income_moment, outside_moment, mushy_moment, firm_moment],
)
pyblp.options.verbose = False

Solving the problem ...

Micro Moments:
Observed               Moment                           Part                   Dataset       Observations  Markets
---------  ------------------------------  ------------------------------  ----------------  ------------  -------
+7.90E+00     E[log_income_i | j > 0]         E[log_income_i | j > 0]       Income Survey        100          2   
+2.80E-01         P(k = 0 | j > 0)                P(k = 0 | j > 0)         Diversion Survey      200          2   
+3.10E-01  P(mushy_j and mushy_k | j > 0)  P(mushy_j and mushy_k | j > 0)  Diversion Survey      200          2   
+3.50E-01    P(firm_j = firm_k | j > 0)      P(firm_j = firm_k | j > 0)    Diversion Survey      200          2   

Nonlinear Coefficient Initial Values:
Sigma:      1        mushy     prices    |   Pi:    log_income
------  ---------  ---------  ---------  |  ------  ----------
  1     +2.00E+00                        |    1     -3.00E-01 
mushy   +0.00E+00  +4.00E+00             |

The estimated nesting parameter fairly low but nonzero, so the new micro statistic seems to suggest some but not a lot of unobserved firm-specific preferences. Parameters in $\Sigma$ are estimated to be a bit lower, presumably because previously they were picking up some unmodeled within-firm preferences.  Let's re-run the counterfactual.

In [ ]:
counterfactual_data['new_shares'] = rcnl_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['rcnl_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,mushy,prices,shares,new_prices,new_shares,iv_change,mushy_change,rc_change,micro_change,rcnl_change
24,F1B04,1,0.078,6.443e-03,0.039,2.724e-02,223.638,223.522,285.128,314.014,322.842
25,F1B06,1,0.141,1.413e-01,0.141,1.338e-01,-1.450,-1.478,-1.621,-4.934,-5.317
26,F1B07,1,0.073,8.789e-02,0.073,8.216e-02,-1.450,-1.478,-1.850,-6.303,-6.523
27,F1B09,0,0.077,6.621e-03,0.077,6.565e-03,-1.450,-1.438,-1.808,-0.750,-0.847
28,F1B11,0,0.167,5.427e-02,0.167,5.396e-02,-1.450,-1.438,-1.496,-0.482,-0.574
29,F1B13,0,0.092,2.198e-02,0.092,2.180e-02,-1.450,-1.438,-1.759,-0.700,-0.796
30,F1B17,1,0.154,1.055e-02,0.154,1.001e-02,-1.450,-1.478,-1.575,-4.696,-5.104
31,F1B30,0,0.150,1.310e-03,0.150,1.302e-03,-1.450,-1.438,-1.556,-0.526,-0.619
32,F1B45,0,0.147,1.052e-02,0.147,1.046e-02,-1.450,-1.438,-1.568,-0.534,-0.628
33,F2B05,0,0.099,5.907e-02,0.099,5.871e-02,-1.450,-1.438,-1.735,-0.677,-0.603


Because we've estimated a nontrivial amount of firm-specific preferences, we get more within-firm substitution than before. The cannibalization estimates in particular are larger compared to substitution from the firm's competitors, as we might expect.